# Fiddler Model Onboarding With Pandas Guide

## Goal

This guide demonstrates how to onboard a model with fiddler using a pandas dataframe.


### Getting Started

1. Connect to Fiddler
2. Load a Data Sample into a Pandas Dataframe
3. Define the Model Specifications
4. Set the Model Task
5. Create the Model Schema from Dataframe
6. Create a Model


# 0. Imports

In [ ]:
%pip install -q fiddler-client

import time as time

import pandas as pd
import fiddler as fdl

print(f'Running Fiddler Python client version {fdl.__version__}')

## 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using the Fiddler Python client.


---


**We need a couple pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your authorization token

Your authorization token can be found by navigating to the **Credentials** tab on the **Settings** page of your Fiddler environment.

In [ ]:
URL = ''  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai').
TOKEN = ''

Constants for this example notebook, change as needed to create your own versions

In [ ]:
PROJECT_NAME = 'quickstart_examples'  # If the project already exists, the notebook will create the model under the existing project.
MODEL_NAME = 'bank_churn_simple_monitoring'

STATIC_BASELINE_NAME = 'baseline_dataset'
ROLLING_BASELINE_NAME = 'rolling_baseline_1week'

# Sample data hosted on GitHub
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_data_sample.csv'
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_production_data.csv'

Now just run the following to connect to your Fiddler environment.

In [ ]:
fdl.init(url=URL, token=TOKEN)

#### 1.a Create New or Load Existing Project

Once you connect, you can create a new project by specifying a unique project name in the fld.Project constructor and calling the `create()` method. If the project already exists, it will load it for use.

In [ ]:
project = fdl.Project.get_or_create(name=PROJECT_NAME)

print(f'Using project with id = {project.id} and name = {project.name}')

You should now be able to see the newly created project in the Fiddler UI.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/simple_monitoring_1.png" />
        </td>
    </tr>
</table>

## 2. Load a Data Sample

In this example, we'll be considering the case where we're a bank and we have **a model that predicts churn for our customers**.
  
In order to get insights into the model's performance, **Fiddler needs a small sample of data** to learn the schema of incoming data.

In [ ]:
sample_data_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
column_list  = sample_data_df.columns
sample_data_df

## 3. Define the Model Specifications

In order to create a model in Fiddler, create a ModelSpec object with information about what each column of your data sample should used for.

Fiddler supports four column types:
1. **Inputs**
2. **Outputs** (Model predictions)
3. **Target** (Ground truth values)
4. **Metadata**

In [ ]:
input_columns = list(
    column_list.drop(['predicted_churn', 'churn', 'customer_id', 'timestamp'])
)

In [ ]:
model_spec = fdl.ModelSpec(
    inputs=input_columns,
    outputs=['predicted_churn'],
    targets=['churn'],  # Note: only a single Target column is allowed, use metadata columns and custom metrics for additional targets
    metadata=['customer_id', 'timestamp'],
)

If you have columns in your ModelSpec which denote **prediction IDs or timestamps**, then Fiddler can use these to power its analytics accordingly.

Let's call them out here and use them when configuring the Model in step 5.

In [ ]:
id_column = 'customer_id'
timestamp_column = 'timestamp'

## 4. Set the Model Task

Fiddler supports a variety of model tasks. In this case, we're adding a binary classification model.

For this, we'll create a ModelTask object and an additional ModelTaskParams object to specify the ordering of our positive and negative labels.

*For a detailed breakdown of all supported model tasks, click [here](https://docs.fiddler.ai/technical-reference/python-client-guides/explainability/model-task-examples).*

In [ ]:
model_task = fdl.ModelTask.BINARY_CLASSIFICATION

task_params = fdl.ModelTaskParams(target_class_order=['no', 'yes'])

## 5. Create a Model Schema from the Dataframe

Create a ModelSchema from the Dataframe

In [ ]:
model_schema = fdl.utils.column_generator.create_columns_from_df(sample_data_df)

Review the resulting model's schema. For e.g., check for min/max range for the returned numeric columns.
Once the results are veried, create the model as shown below.

In [ ]:
model = fdl.Model(
    name=MODEL_NAME,
    project_id=project.id,
    schema=model_schema,
    spec=model_spec,
    task=model_task,
    task_params=task_params
)

model.create()

On the project page, you should now be able to see the newly onboarded model with its model schema.

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_3.png?raw=true" />
        </td>
    </tr>
</table>

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_4.png?raw=true" />
        </td>
    </tr>
</table>

**What's Next?**

Try the [LLM Monitoring - Quick Start Notebook](https://docs.fiddler.ai/tutorials-and-quick-starts/llm-and-genai/simple-llm-monitoring)

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.